## Part 1

### 1. Identify the Target
DONE

### 2. Interact with the Page-Sorting
I can trigger the sorting change directly by adding "oldest" after "date" in "sort=". Changing the url from https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~0~0 to https://sfbay.craigslist.org/search/zip?sort=dateoldest#search=1~gallery~0~0.

When I change the sort order, I made a get request, because I did not provide any information to the server but just receive information.

The variable associate with sorting is "sort", we can change sorting by change values after sort=https://sfbay.craigslist.org/search/zip?sort=dateoldest#search=1~gallery~0~0.

### 3. Interact with the Page-Pagination
To change the page using url, I can change the first number after gallery in the search variable. To change from the first to the 2nd page, change the variable from search=1\~gallery\~0\~1 to search=1\~gallery\~1\~1.

The variable associate with page change is serach. When change the 1st number after gallery, the we change page being viewed. When change the 2nd number after gallery, we scroll down or up the same page.

### 4. Fetch Listing URLs

In [247]:
from bs4 import BeautifulSoup
import requests
import re
import time
import os
import pandas as pd
from pprint import pprint

In [434]:
# load and parse the page in bs
url = 'https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~0'
headers = {'User-Agent':'Mozilla/5.0'}
page1 = requests.get(url,headers)
soup1 = BeautifulSoup(page1.content,'html.parser')

1. I use selector \<a\> and get href in which stored the links to individual listing pages

2. An alternative selection method is to select 'div.cl-gallery' and then find its next sibling using ".next_sibling"

In [435]:
# print(soup1.prettify())
# get all elements with tag a
frees1 = soup1.select('a')
# frees1 includes the first 360 items, so get the first 250 items as needed
frees1 = frees1[2:252]
len(frees1)

250

Since the element we extract above includes items beyond the first page (there are 360 items, so items to the 3rd page), we do not need to find the link for next page and navigate to the 2nd and 3rd page. However, if navigating pages are needed, I will change the first number after gallery (in url) to 2 and 3 to get my 2nd and 3rd url, and then get the links for items on that page. Here, without page navigation, my strategy to find all tag a element and get the href from them. Since the request returns the first 360 items, I select the first 250 items. For each of the tag a element, I get the href in which stores the link we need. As a result, there are 250 links in the list

In [436]:
links = []
for free in frees1:
    links.append(free.get('href'))
len(links)

250

In [437]:
links

['https://sfbay.craigslist.org/eby/zip/d/oakland-curbside-pickup-tuesday-6-til-pm/7715041240.html',
 'https://sfbay.craigslist.org/eby/zip/d/antioch-10-laying-hens-free/7715041015.html',
 'https://sfbay.craigslist.org/eby/zip/d/san-ramon-free-sideboard-nightstand-for/7715040980.html',
 'https://sfbay.craigslist.org/eby/zip/d/richmond-free-wooden-queen-bed-frame/7715040471.html',
 'https://sfbay.craigslist.org/eby/zip/d/oakland-ikea-billy-bookshelf-doors-19x90/7715038514.html',
 'https://sfbay.craigslist.org/nby/zip/d/sebastopol-plant-stand/7715038437.html',
 'https://sfbay.craigslist.org/eby/zip/d/discovery-bay-free-wood-chip-mulch/7711252862.html',
 'https://sfbay.craigslist.org/sby/zip/d/san-jose-books/7712487821.html',
 'https://sfbay.craigslist.org/nby/zip/d/santa-rosa-free-pallets-wood/7711471768.html',
 'https://sfbay.craigslist.org/eby/zip/d/martinez-free-crib/7715035794.html',
 'https://sfbay.craigslist.org/sfc/zip/d/san-francisco-clothing-racks/7715035114.html',
 'https://sfba

### 5. Save HTML Pages

In [438]:
# get the ids of items
id_pattern = r'[\',./\-:a-z0-9]*/([0-9]*).html'
id_num = re.findall(id_pattern,str(links))
len(id_num) == len(links) # the number of id matches the number of links

True

In [439]:
# store till 88th file
for id_n,link in zip(id_num[0:89],links[0:89]):
    response = requests.get(link)
    with open(f'{id_n}.html','w',encoding = 'UTF-8') as file:
        file.write(response.text)
    time.sleep(5.2)

In [440]:
# store the rest html files
for id_n,link in zip(id_num[89:],links[89:]):
    response = requests.get(link)
    with open(f'{id_n}.html','w',encoding = 'UTF-8') as file:
        file.write(response.text)
    time.sleep(5.2)

In [105]:
#trial1 = requests.get('https://sfbay.craigslist.org/sfc/zip/d/san-francisco-photography-on-mixtiles/7714834960.html')
with open(f'{id_num[0]}.html','w',encoding = 'UTF-8') as file:
    file.write(trial1.text)

## PART 2

### 1 & 2

### Create a Function to Get All Info

In [ ]:
# create a function to return all information
def post_info(afile):
    # soup
    soup2 = BeautifulSoup(afile)
    
    #title
    title = soup2.select_one('#titletextonly').text.strip()
    
    # 1st image
    # check if image exists
    image = soup2.select('img')
    if len(image) != 0:
        image = soup2.select_one('img').get('src')
    else:
        image ='No Image shown.'

        
    # description
    description = soup2.select_one('#postingbody>div').next_sibling.strip()
    
    # post_id
    post_id = soup2.select_one('div.postinginfos>p')
    post_id_pattern = r'[^0-9]*([0-9]*)</p>'
    post_id = re.findall(post_id_pattern,str(post_id))
    post_id = post_id[0]
    post_id
    
    # post date
    post_date = soup2.select_one('#display-date>time').get('datetime')
    
    # last update date
    update_dates = soup2.select('div.postinginfos>p')[2]
    # check if last update date exist
    if len(update_dates) != 0:
        update_pattern = r'[^0-9]*([-:0-9A-Z]*)".*'
        update_date = re.findall(update_pattern,str(update_dates))
        update_date = update_date[0]

    else:
        update_date = 'Post not updated since post.'
    
    # create a dataframe with all info
    all_info = {
        'title':title,
        'image':image,
        'description':description,
        'post_id':post_id,
        'post_date':post_date,
        'last_update_date':update_date
        }
    pprint(all_info)

    

### Read Saved HTML Files and Extract Info

In [441]:
# get the current working directory
directory = os.getcwd()

df = pd.DataFrame()

filenames=[]

# Loop through each file in the directory
for filename in os.listdir(directory):
    
    # Check if the file ends with .html
    if filename.endswith(".html"):

        filenames.append(filename)


In [448]:
# info till 116
for filename in filenames[0:116]:
    # Read file to string
    with open(filename, 'r', encoding='utf-8') as file:
        html = file.read()

    post_info(html)

{'description': "Please send me your number and I'll send you a text back. No "
                'Number no response.',
 'image': 'https://images.craigslist.org/01717_aun3vlamndS_0t20CI_600x450.jpg',
 'last_update_date': '2024-02-06T08:23:38-0800',
 'post_date': '2024-02-03T11:19:15-0800',
 'post_id': '7714105140',
 'title': 'FREE electrical conduits pipes'}
{'description': 'Working and clean condition, from a pet free and non smoking '
                'house.',
 'image': 'https://images.craigslist.org/01414_kocNH60uVG6_0CI0t2_600x450.jpg',
 'last_update_date': '2024-02-05T23:32:39-0800',
 'post_date': '2024-01-24T13:00:05-0800',
 'post_id': '7710855801',
 'title': 'Free Sofa'}
{'description': 'Six vintage cardboard barrels with tension strap metal lids. '
                'Probably from the ‘40s or 50s',
 'image': 'https://images.craigslist.org/00L0L_5791jbsuPlr_0CI0t2_600x450.jpg',
 'last_update_date': '',
 'post_date': '2024-02-05T19:28:14-0800',
 'post_id': '7714884073',
 'title': 'V

In [451]:
# info 117-146
for filename in filenames[117:146]:
    # Read file to string
    with open(filename, 'r', encoding='utf-8') as file:
        html = file.read()

    post_info(html)

{'description': 'Lightly used, has springs, is comfy with a foam topper.',
 'image': 'https://images.craigslist.org/00s0s_5f0IEOHuWni_0t20CI_600x450.jpg',
 'last_update_date': '',
 'post_date': '2024-02-06T07:47:37-0800',
 'post_id': '7714980066',
 'title': 'Hide-a-bed mattress. 60x72x5'}
{'description': 'Grey desk chair',
 'image': 'https://images.craigslist.org/01111_bl8NjT2TTPj_0lM0t2_600x450.jpg',
 'last_update_date': '',
 'post_date': '2024-02-05T16:27:11-0800',
 'post_id': '7714844663',
 'title': 'Desk chair'}
{'description': 'REI brand camping pad to go under a sleeping bag like a '
                'thermarest. The pad originally came with two valves. One of '
                'the valves fell off and I repaired the hole with Gear-Aid '
                'tenacious tape. However the second valve is now starting to '
                'come apart and the mattress does not hold air. There is '
                'internal padding inside the sleeping pad, so there is some '
               

In [454]:
# info 147-240
for filename in filenames[147:240]:
    # Read file to string
    with open(filename, 'r', encoding='utf-8') as file:
        html = file.read()

    post_info(html)

{'description': 'Free outdoor chairs with cushions (8). Some chairs are in '
                'good condition and some are not. Must take all 8 and not just '
                'the cushions. Please text',
 'image': 'https://images.craigslist.org/00V0V_iNCoIykbkqx_0t20CI_600x450.jpg',
 'last_update_date': '',
 'post_date': '2024-02-06T08:14:34-0800',
 'post_id': '7714991290',
 'title': 'Free outdoor chairs with cushions'}
{'description': 'IKEA HAUGSVÄR',
 'image': 'https://images.craigslist.org/00Y0Y_c9XYBg3tuRz_0CI0CI_600x450.jpg',
 'last_update_date': '',
 'post_date': '2024-02-06T08:01:51-0800',
 'post_id': '7714985683',
 'title': 'Twin mattress - 3 years old'}
{'description': 'Easy to put together but I no longer have the manual for it. '
                'Please respond only if you are serious about picking it up. '
                'If the add is still up then it is still available.',
 'image': 'https://images.craigslist.org/00e0e_17Agn9JxY7X_07K0ak_600x450.jpg',
 'last_update_date': 

In [455]:
# info after 241
for filename in filenames[241:]:
    # Read file to string
    with open(filename, 'r', encoding='utf-8') as file:
        html = file.read()

    post_info(html)

{'description': 'Crack in top right loose piece can be glued or cut smaller',
 'image': 'https://images.craigslist.org/01111_eDeUqUeOjAh_0t20CA_600x450.jpg',
 'last_update_date': '2024-02-05T19:23:16-0800',
 'post_date': '2024-01-24T16:38:53-0800',
 'post_id': '7710929494',
 'title': 'Large mirror'}
{'description': '1000 piece puzzle 🧩',
 'image': 'https://images.craigslist.org/00H0H_1QSTF3uXEEt_0tT0t2_600x450.jpg',
 'last_update_date': '',
 'post_date': '2024-02-05T21:09:01-0800',
 'post_id': '7714897627',
 'title': '1000 piece puzzle 🧩'}
{'description': '1970s era Craftsman. Works fine. Has new motor',
 'image': 'https://images.craigslist.org/01515_dwU8snNrNRr_0t20CI_600x450.jpg',
 'last_update_date': '',
 'post_date': '2024-02-05T16:48:50-0800',
 'post_id': '7714850184',
 'title': '10" Craftsman tablesaw'}
{'description': 'Diamondback 860Er.  Works just fine.  Easily transportable - '
                'can fold down and slide in the back of any SUV with the hatch '
                't

### Note
3 posts are skipped above because the original post was deleted by the author. It is time consuming to run everything all over again to get the updated 250 items, because some authors delete their items right after they post it. Therefore, I skipped the 3 items that are deleted. The code words fine if all 250 items are not deleted

## Part 3

### 1. Creating and Verifying a The Old Reader Account
Done

### 2. Exploring the Login Mechanism
The name attributed in \<input\> fields are:  
name = "utf8"  
name = "authenticity_token"  
name = "user\[login\]"  
name = "user\[password\]"  
name = "commit"  

Unlike other name inputs, user\[login\] and user\[password\] has no value

### 3. Analyzing Network Traffic for Login Request
The request method was post when I submitted the login form. It uses a post request because I submit some information (my user name and password) to the server to obtain information.

Comparing to the input before log in that we previously analyzed, the payload after login have values for user\[login\] and user\[password\].


### 4. Automating the Login Process

In [457]:
headers3 = {'User-Agent':'Mozilla/5.0'}
url3 = 'https://theoldreader.com/users/sign_in'
page3 = requests.get(url3, headers = headers)
soup3 = BeautifulSoup(page3.content, 'html.parser')

In [488]:
# get authenticity_token value
form = soup3.select_one('#new_user')
input_tk = form.select_one('input[name = authenticity_token]')
token = input_tk.get('value')

# get utf8 value
input_utf = form.select_one('input[name = utf8]')
utf = input_utf.get('value')

# get commit value
input_commit = form.select_one('input[name = commit]')
commit = input_commit.get('value')

In [490]:
# put time sleep
time.sleep(5)

# create a session
session = requests.session()

res = session.post('https://theoldreader.com/users/sign_in',
                  data = {'utf8':utf,
                          'authenticity_token':token,
                          'user[login]':'jiayuanqian@brandeis.edu',
                          'user[password]':'Bax422ddrpractice',
                          'commit':commit},
                  timeout = 20) # timeout is the maximum amount of time the request is allowed to take


### 5. Verifying Successful Login

In [491]:
# get cookies to keep us logged in
cookies = session.cookies.get_dict()
print(cookies)

{'_new_reader_session': 'BAh7CkkiD3Nlc3Npb25faWQGOgZFVEkiJTY2YTE4OTdmY2NlZDM0MjZhMjA3OTY3NDY1OTI1ODZhBjsAVEkiGXdhcmRlbi51c2VyLnVzZXIua2V5BjsAVFsHWwZVOhpNb3BlZDo6QlNPTjo6T2JqZWN0SWQiEVVxNqqoDb%2FWSKK%2FPEkiIiQyYSQwNSRTZklZR1d4cW1qNXhRbHNUZzF2Szl1BjsAVEkiDWxhbmd1YWdlBjsARjoHZW5JIhByZWRpcmVjdF90bwY7AEZJIgYvBjsARkkiEF9jc3JmX3Rva2VuBjsARkkiMUR1MUV4Yi9EVldpc1hWaU9lVXMvTGdBUTBGTTZ5ZXIvUVVHU3p6d3FEUEU9BjsARg%3D%3D--cc46dcc03b3b3259f842223e09a6e798316b73f1', 'i_know_you': 'Judy', 'remember_user_token': 'BAhbB1sGVToaTW9wZWQ6OkJTT046Ok9iamVjdElkIhFVcTaqqA2%2F1kiivzxJIiIkMmEkMDUkU2ZJWUdXeHFtajV4UWxzVGcxdks5dQY6BkVU--3de4be15d0c0f18ce0e55ba4db5790b351ffbb9b', 'signed_at': '1707256723'}


In [513]:
time.sleep(5)
# verify I'm logged in
page4 = session.get('https://theoldreader.com/')
soup4 = BeautifulSoup(page4.content, 'html.parser')


In [512]:
# check if my email is in the html file
soup4.select_one('a.dropdown-toggle').get('title')

'Judy'

The username "Judy" is only available after I log in. Therefore, I succeddfully logged in to the website